<a href="https://colab.research.google.com/github/fdmy2713-dotcom/MSc-in-Data-Science/blob/main/Farah_ADS1_Assessment_2_Clustering_and_Fitting_v1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
"""
Public Transport Usage & Premium Bus Services Analysis
------------------------------------------------------
This script is designed for a presentation report that covers:

- Categorical graph (bar chart)
- Relational graph (line / scatter)
- Statistical graph (box plot + heatmap)
- Clustering graphs (elbow / silhouette + cluster scatter)
- Line fitting (linear regression)

Datasets used (you already uploaded these):
- monthly_ave_daily_pt_ridership.csv
- PremiumBusServicesCSV20241125.csv
"""

'\nPublic Transport Usage & Premium Bus Services Analysis\n------------------------------------------------------\nThis script is designed for a presentation report that covers:\n\n- Categorical graph (bar chart)\n- Relational graph (line / scatter)\n- Statistical graph (box plot + heatmap)\n- Clustering graphs (elbow / silhouette + cluster scatter)\n- Line fitting (linear regression)\n\nDatasets used (you already uploaded these):\n- monthly_ave_daily_pt_ridership.csv\n- PremiumBusServicesCSV20241125.csv\n'

In [ ]:
# ======================
# 1. Imports & Settings
# ======================

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import silhouette_score, mean_absolute_error, r2_score
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split

import warnings

warnings.filterwarnings("ignore")
sns.set(style="whitegrid", context="talk")


In [ ]:
# ======================
# 2. CONFIGURATION
# ======================
# >>>>> EDIT ONLY THIS SECTION TO MATCH YOUR CSV COLUMN NAMES <<<<<

# File paths (keep as is if you run in the same folder / Colab upload)
RIDERSHIP_CSV = "monthly_ave_daily_pt_ridership.csv"
PREMIUM_CSV = "PremiumBusServicesCSV20241125.csv"

# --- Ridership dataset (monthly_ave_daily_pt_ridership.csv) ---
# Assumed typical columns (change to match your file):
RID_YEAR_COL = "Year"            # e.g. 2019, 2020...
RID_MONTH_COL = "Month"          # 1–12 or month name
RID_MODE_COL = "PT_TYPE"         # e.g. MRT / LRT / Bus / Taxi
RID_VALUE_COL = "Average_Daily_Ridership"  # numeric ridership

# --- Premium Bus dataset (PremiumBusServicesCSV20241125.csv) ---
# Again, change names if needed:
PB_SERVICE_COL = "ServiceNo"
PB_OPERATOR_COL = "Operator"
PB_DISTANCE_COL = "Distance"           # numeric distance in km (if available)
PB_FARE_COL = "AM_Peak_Fare"          # numeric fare for clustering / stats

In [ ]:
# ======================
# 3. Data Loading & Cleaning
# ======================

def load_ridership(path: str) -> pd.DataFrame:
    """
    Load the monthly average daily ridership dataset and standardise column names.
    Expected columns: year, month, mode, ridership (long format).
    """
    df = pd.read_csv(path)

    # Rename columns to internal standard
    rename_map = {
        RID_YEAR_COL: "year",
        RID_MONTH_COL: "month",
        RID_MODE_COL: "mode",
        RID_VALUE_COL: "ridership",
    }
    df = df.rename(columns=rename_map)

    # Coerce numeric types
    df["year"] = pd.to_numeric(df["year"], errors="coerce")
    df["ridership"] = pd.to_numeric(df["ridership"], errors="coerce")

    # Handle month: if numeric OK; if string, convert to month number
    if not np.issubdtype(df["month"].dtype, np.number):
        df["month"] = pd.to_datetime(
            df["month"].astype(str), format="%b", errors="coerce"
        ).dt.month

    df = df.dropna(subset=["year", "month", "mode", "ridership"])
    df = df.drop_duplicates().reset_index(drop=True)
    return df


def load_premium(path: str) -> pd.DataFrame:
    """
    Load the premium bus services dataset and standardise important columns.
    """
    df = pd.read_csv(path)

    rename_map = {
        PB_SERVICE_COL: "service",
        PB_OPERATOR_COL: "operator",
        PB_DISTANCE_COL: "distance",
        PB_FARE_COL: "fare",
    }
    df = df.rename(columns=rename_map)

    df["distance"] = pd.to_numeric(df["distance"], errors="coerce")
    df["fare"] = pd.to_numeric(df["fare"], errors="coerce")

    df = df.dropna(subset=["service", "operator"])
    df = df.drop_duplicates().reset_index(drop=True)
    return df


In [ ]:
# ======================
# 4. Categorical / Relational / Statistical Graphs
# ======================

def plot_bar_ridership_by_mode(rid_df: pd.DataFrame) -> None:
    """
    Categorical graph (for rubric: Categorical Graph Quality + Function).
    Total monthly ridership summed by mode.
    """
    summary = rid_df.groupby("mode")["ridership"].sum().sort_values(ascending=False)

    plt.figure(figsize=(10, 6))
    sns.barplot(x=summary.index, y=summary.values)
    plt.title("Total Ridership by Mode")
    plt.xlabel("Public Transport Mode")
    plt.ylabel("Total Average Daily Ridership")
    plt.tight_layout()
    plt.show()


def plot_bar_premium_by_operator(pb_df: pd.DataFrame) -> None:
    """
    Additional categorical graph using premium bus data.
    Shows how many premium services each operator runs.
    """
    counts = pb_df.groupby("operator")["service"].nunique().sort_values(ascending=False)

    plt.figure(figsize=(10, 6))
    sns.barplot(x=counts.index, y=counts.values)
    plt.title("Number of Premium Bus Services by Operator")
    plt.xlabel("Operator")
    plt.ylabel("Number of Services")
    plt.tight_layout()
    plt.show()


def plot_line_ridership_trend(rid_df: pd.DataFrame) -> None:
    """
    Relational / line graph (for rubric: Relational Graph Quality + Function).
    Monthly ridership trend over time for each mode.
    """
    tmp = rid_df.copy()
    tmp["year_month"] = pd.to_datetime(
        tmp["year"].astype(int).astype(str) + "-" + tmp["month"].astype(int).astype(str) + "-01"
    )

    monthly = (
        tmp.groupby(["year_month", "mode"])["ridership"]
        .mean()
        .reset_index()
        .sort_values("year_month")
    )

    plt.figure(figsize=(12, 6))
    sns.lineplot(
        data=monthly,
        x="year_month",
        y="ridership",
        hue="mode",
        marker="o",
    )
    plt.title("Monthly Average Daily Ridership Trend by Mode")
    plt.xlabel("Month")
    plt.ylabel("Average Daily Ridership")
    plt.xticks(rotation=45)
    plt.tight_layout()
    plt.show()


def plot_box_ridership_by_mode(rid_df: pd.DataFrame) -> None:
    """
    Statistical graph (box plot) (for rubric: Statistical Graph Quality + Function).
    Distribution of ridership by mode.
    """
    plt.figure(figsize=(10, 6))
    sns.boxplot(data=rid_df, x="mode", y="ridership")
    plt.title("Ridership Distribution by Mode (Box Plot)")
    plt.xlabel("Mode of Transport")
    plt.ylabel("Average Daily Ridership")
    plt.tight_layout()
    plt.show()


def plot_corr_heatmap(rid_df: pd.DataFrame) -> None:
    """
    Statistical graph (heatmap of correlations).
    Uses pivoted wide-format monthly ridership by mode.
    """
    pivot = (
        rid_df.pivot_table(
            index=["year", "month"], columns="mode", values="ridership", aggfunc="mean"
        )
        .reset_index()
        .drop(columns=["year", "month"])
    )

    corr = pivot.corr(method="pearson")

    plt.figure(figsize=(8, 6))
    sns.heatmap(
        corr,
        annot=True,
        fmt=".2f",
        cmap="coolwarm",
        vmin=-1,
        vmax=1,
        square=True,
    )
    plt.title("Correlation Heatmap Between Modes (Monthly Ridership)")
    plt.tight_layout()
    plt.show()


In [ ]:
# ======================
# 5. K-Means Clustering & Plots
# ======================

def prepare_clustering_features(rid_df: pd.DataFrame) -> (np.ndarray, pd.DataFrame):
    """
    Prepare features for clustering: monthly ridership by mode (wide format).
    Each row = a (year, month) combination, columns = modes.
    """
    wide = rid_df.pivot_table(
        index=["year", "month"],
        columns="mode",
        values="ridership",
        aggfunc="mean",
    ).fillna(0.0)

    feat_df = wide.reset_index()  # keep year & month for later use

    scaler = StandardScaler()
    X_scaled = scaler.fit_transform(feat_df.drop(columns=["year", "month"]))

    return X_scaled, feat_df


def plot_elbow_curve(X_scaled: np.ndarray, max_k: int = 10) -> None:
    """
    Elbow plot (for rubric: Statistical Graph Quality + Clustering Function).
    Shows inertia vs k.
    """
    inertias = []
    k_values = range(2, max_k + 1)

    for k in k_values:
        km = KMeans(n_clusters=k, random_state=42)
        km.fit(X_scaled)
        inertias.append(km.inertia_)

    plt.figure(figsize=(8, 5))
    plt.plot(list(k_values), inertias, marker="o")
    plt.xlabel("Number of Clusters (k)")
    plt.ylabel("Inertia (Within-Cluster SSE)")
    plt.title("Elbow Plot for K-Means Clustering")
    plt.xticks(list(k_values))
    plt.tight_layout()
    plt.show()


def plot_silhouette_scores(X_scaled: np.ndarray, max_k: int = 10) -> None:
    """
    Silhouette plot (for rubric: Statistical Graph Function + Clustering Quality).
    Average silhouette score vs k.
    """
    scores = []
    k_values = range(2, max_k + 1)

    for k in k_values:
        km = KMeans(n_clusters=k, random_state=42)
        labels = km.fit_predict(X_scaled)
        scores.append(silhouette_score(X_scaled, labels))

    plt.figure(figsize=(8, 5))
    plt.plot(list(k_values), scores, marker="o")
    plt.xlabel("Number of Clusters (k)")
    plt.ylabel("Average Silhouette Score")
    plt.title("Silhouette Scores for K-Means Clustering")
    plt.xticks(list(k_values))
    plt.tight_layout()
    plt.show()


def perform_kmeans(X_scaled: np.ndarray, feat_df: pd.DataFrame, k: int = 3) -> pd.DataFrame:
    """
    Run K-Means and return feature dataframe with cluster labels.
    """
    km = KMeans(n_clusters=k, random_state=42)
    labels = km.fit_predict(X_scaled)

    feat_df = feat_df.copy()
    feat_df["cluster"] = labels

    print("\nCluster sizes:")
    print(feat_df["cluster"].value_counts().sort_index())
    return feat_df


def plot_clusters(feat_df: pd.DataFrame) -> None:
    """
    Relational graph:
    Scatter ridership of one mode vs another, coloured by cluster.
    Picks first two mode columns automatically.
    """
    # Detect mode columns (everything except year, month, cluster)
    mode_cols = [c for c in feat_df.columns if c not in ["year", "month", "cluster"]]
    if len(mode_cols) < 2:
        print("Not enough modes for 2D cluster scatter.")
        return

    x_col, y_col = mode_cols[0], mode_cols[1]

    plt.figure(figsize=(10, 6))
    sns.scatterplot(
        data=feat_df,
        x=x_col,
        y=y_col,
        hue="cluster",
        palette="tab10",
    )
    plt.title(f"Clustered Monthly Ridership ({x_col} vs {y_col})")
    plt.xlabel(f"{x_col} Ridership")
    plt.ylabel(f"{y_col} Ridership")
    plt.legend(title="Cluster")
    plt.tight_layout()
    plt.show()


In [ ]:
# ======================
# 6. Linear Regression Fitting
# ======================

def fit_linear_trend(rid_df: pd.DataFrame):
    """
    Fitting function (for rubric: Fitting Function / Quality / Accuracy).
    Simple model: yearly average ridership vs year.
    """
    yearly = (
        rid_df.groupby("year")["ridership"]
        .mean()
        .reset_index()
        .sort_values("year")
    )

    X = yearly[["year"]].values
    y = yearly["ridership"].values

    X_train, X_test, y_train, y_test = train_test_split(
        X, y, test_size=0.2, random_state=42
    )

    model = LinearRegression()
    model.fit(X_train, y_train)

    y_pred = model.predict(X_test)

    mae = mean_absolute_error(y_test, y_pred)
    r2 = r2_score(y_test, y_pred)

    print("\n=== Linear Regression Results ===")
    print(f"Intercept: {model.intercept_:.3f}")
    print(f"Slope (per year): {model.coef_[0]:.3f}")
    print(f"Mean Absolute Error (MAE): {mae:.3f}")
    print(f"R² Score: {r2:.3f}")

    return model, yearly


def plot_linear_fit(model: LinearRegression, yearly: pd.DataFrame) -> None:
    """
    Relational/Statistical graph:
    Scatter of yearly ridership + fitted regression line.
    """
    X_all = yearly[["year"]].values
    y_all = yearly["ridership"].values
    y_fit = model.predict(X_all)

    plt.figure(figsize=(10, 6))
    plt.scatter(yearly["year"], y_all, label="Actual", alpha=0.7)
    plt.plot(yearly["year"], y_fit, color="red", label="Fitted trend", linewidth=2)
    plt.title("Yearly Average Ridership with Linear Trend Line")
    plt.xlabel("Year")
    plt.ylabel("Average Daily Ridership")
    plt.legend()
    plt.tight_layout()
    plt.show()


In [ ]:
# ======================
# 7. MAIN: End-to-End Workflow
# ======================

def main():
    # 1. Load datasets
    rid_df = load_ridership(RIDERSHIP_CSV)
    pb_df = load_premium(PREMIUM_CSV)

    print("Ridership data shape:", rid_df.shape)
    print("Premium bus data shape:", pb_df.shape)

    # 2. Categorical graphs
    plot_bar_ridership_by_mode(rid_df)      # i) histogram/bar/pie style
    plot_bar_premium_by_operator(pb_df)

    # 3. Relational graph
    plot_line_ridership_trend(rid_df)       # ii) line/scatter graph

    # 4. Statistical graphs (box + heatmap)
    plot_box_ridership_by_mode(rid_df)      # iii) statistical: box plot
    plot_corr_heatmap(rid_df)               # iii) statistical: heatmap

    # 5. Clustering – elbow & silhouette + final cluster scatter
    X_scaled, feat_df = prepare_clustering_features(rid_df)
    plot_elbow_curve(X_scaled, max_k=8)         # iv) elbow plot
    plot_silhouette_scores(X_scaled, max_k=8)   # iv) silhouette plot

    # Choose k based on elbow/silhouette (for demo, pick k=3)
    clustered = perform_kmeans(X_scaled, feat_df, k=3)
    plot_clusters(clustered)

    # 6. Fitting – linear regression on yearly trend
    model, yearly = fit_linear_trend(rid_df)
    plot_linear_fit(model, yearly)

    # 7. Example prediction (for presentation)
    future_year = yearly["year"].max() + 1
    future_pred = model.predict(np.array([[future_year]]))[0]
    print(
        f"\nPredicted average daily ridership for year {future_year}: "
        f"{future_pred:,.0f}"
    )


if __name__ == "__main__":
    main()
